In [23]:
import torch
import numpy as np

In [9]:
# tensor,张量,是一个多维数组
#推荐第四种创建方法,在内存管理时,会共享一大块内存,计算更快
#1.用标准的数据格式来初始化tensor
data = [[1, 2],[3, 4]]# python list
x_data0 = torch.tensor(data)
x_data1 = torch.tensor((1, 2, 3))  #tuple
#x_data2 = torch.tensor({'a':5})  # RuntimeError: Could not infer dtype of dict
print(x_data0)

tensor([[1, 2],
        [3, 4]])


In [11]:
#2.用torch的标准函数直接生成特殊的tensor
data0 = torch.ones(1, 2, 3)  #全1矩阵
data1 = torch.zeros(1, 3, 4)  # 全0矩阵
data2 = torch.randn(3, 4, 5)  # 正态分布的随机数
data3 = torch.eye(4, 5)  # 对角阵
data4 = torch.randint(5,(2,10))  # 随机整数
print(data0)
print(data1)
print(data2)
print(data3)
print(data4)

tensor([[[1., 1., 1.],
         [1., 1., 1.]]])
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
tensor([[[-1.1371, -0.3871,  0.5021, -2.9149, -1.6295],
         [ 0.5967, -0.0721, -0.7843, -0.8640, -0.0751],
         [ 0.0481, -0.0503,  0.2114, -0.9993, -0.2645],
         [ 0.8848,  0.9769, -0.7354,  2.2181, -0.1744]],

        [[-1.9941, -1.0963,  0.7392, -0.0870,  0.8535],
         [-1.1744,  0.0852, -0.6836,  0.4369,  1.0487],
         [-0.9459,  0.3791, -0.0123,  0.6430,  1.0226],
         [-0.8113,  1.6142,  1.5126,  0.9987,  0.2323]],

        [[-0.9552, -0.5129,  0.5832,  0.8393, -0.0778],
         [ 0.6723,  0.9066,  1.0917,  0.7279,  1.7121],
         [-0.7453,  0.1757, -1.6120, -0.5248,  0.4208],
         [ 0.1334, -1.1884, -0.5246,  0.4482,  1.1182]]])
tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])
tensor([[1, 2, 1, 2, 1, 2, 4, 3, 0, 0],
        [4, 3, 2, 0, 1, 4, 0, 

In [22]:
#3.仿照其他tensor生成
data0 = torch.Tensor([1, 2, 3])
data1 = torch.ones_like(data0) # shape参考data0,device也参考(把tensor放到GPU上)
data2 = torch.empty_like(data0)
print(data2)

data0 = data0.cuda()
print(data0)
data1 = torch.ones_like(data0)
data1

tensor([0.0000, 4.4766, 3.0000])
tensor([1., 2., 3.], device='cuda:0')


tensor([1., 1., 1.], device='cuda:0')

In [29]:
#4.从numpy生成
np_array = np.array([1, 2, 3])
tensor_from_numpy1 = torch.from_numpy(np_array)
#tensor_from_numpy2 = torch.Tensor(np_array)  # deepcopy
np_array[0] = 100
data_numpy = tensor_from_numpy1.numpy()

# 下面是一个接口.可以查看具体的内存标识号
def numpy_with_torch_tensor():
  ndarray = np.array([1, 3, 4])
  tensor = torch.tensor(ndarray)
  tensor_from_numpy = torch.from_numpy(ndarray)
  
  print("numpy data_ptr: ", ndarray.ctypes.data)
  print("torch data_ptr: ", tensor.data_ptr())
  print("tensor_from numpy data_ptr: ", tensor_from_numpy.data_ptr())

  ndarray_from_torch = tensor_from_numpy.numpy()
  print("ndarray_from_torch data_ptr: ",   ndarray_from_torch.ctypes.data)

numpy_with_torch_tensor()

numpy data_ptr:  2628505150864
torch data_ptr:  3694376586368
tensor_from numpy data_ptr:  2628505150864
ndarray_from_torch data_ptr:  2628505150864


In [31]:
#二.数据类型转换
tensor = torch.ones(4, 5)
# tensor.type()
def tensor_to_demo():
    tensor = torch.ones(4, 5)
    tensor0 = tensor.to(torch.int32)
    tensor1 = tensor.to(tensor0)
    # float16: 1bit(符号位) + 5bit(指数位) + 10bit(尾数) 
    # bfloat16：1bit(符号位) + 8bit(指数位) + 7bit(尾数)
    # BF16背后的想法是通过降低数字的精度来减少计算能力和将张量相乘所需的能源消耗
    tensor2 = tensor.to(torch.bfloat16) # 数据类型转化

'torch.FloatTensor'

In [44]:
# 三.device转化
def tensor_device_demo():
  if torch.cuda.is_available():
    device = torch.device("cuda:0")
  else:
    device = torch.device("cpu")

  tensor = torch.randn(4, 5)
  tensor_0 = tensor.to(device) # H2D   Host to device ,从主机内存复制数据到设备内存
  tensor_1 = tensor.to('cpu')  #D2H      offload
  tensor_2 = tensor.cuda()
  tensor_3 = tensor.to(tensor_0)

In [59]:
# 四.怎么看两个tensor是不是占用用一个内存
def reshape_demo():
    data0 = torch.randn(4, 5)
    data1 = data0.reshape(5, 4)
    print(data0)
    print(data1)
#     print(data0.storage())  # 但是只能看表象
#     print(data1.storage())
    print(data0.data_ptr())
    print(data1.data_ptr())  #  看指针,data0和data1是同一个tensor


    
# Pytorch 默认按行存储
# stride,步长,是meta数据中重要的信息之一,按照元素级别来对应,numpy是按照Byte级别对应.理论来说,(2,3,4)的stride为(12,4,1)
def reshape_view():
    data0 = torch.randn(4, 5)
    data1 = data0.view(5, 4)
#     print(data0.stride())  # (5,1)
#     print(data1.stride())  #(4,1)
#     print(data0.data_ptr())  #3694376847104
#     print(data1.data_ptr())  #3694376847104
    
"""
那么,view和reshape肉眼看上去,效果一样,有什么区别?
大部分情况下二者并无区别,只有针对不连续的数据才会发生区别,使用view会更加安全,永远不会重新搞一份数据出来
当原来数据不连续时view会失败(比如,对data转秩之后,再view会失败),reshape当原来数据连续直接使用,反之重新copy一份数据,也就是说reshape永远可以运行成功.
"""
    
def reshape_transpose():
    data0 = torch.randn(4, 5)
    data1 = data0.T  # (5,4),此处虽然做了转秩,但还是和data0共享同一份数据.Pytorch里做转秩数据不会真正的搬迁,但是stride会变化,stride对应做转秩
    print(data0.stride())  # (5,1)
    print(data1.stride())  # (1,5), 此时按理来说应该是(4,1)
    print(data0.is_contiguous())  #True
    print(data1.is_contiguous())  #False,不连续
    
    
    data2 = data1.contiguous()
    print(data2.is_contiguous())  #但是此时data2就不是和data1共享同一份数据量,发生了搬迁,相当于整块copy了一份
    
    
"""
一个data的stride和shape是有一一对应的关系的,即shape(2,3,4)的stride为(12,4,1).
若不满足这种关系,则发生了数据不连续(data.is_contiguous()),导致做计算式不可以成片的搬迁使用数据,导致效率降低
此时需要手动使其连续(data.contiguous())
那么在整个Pytorch算子中,只有transpose和premute操作会发生不连续的情况,因为往往在后面会手动使其连续
"""
if __name__ == '__main__':
#     reshape_demo()
#     reshape_view()
    reshape_transpose()

(5, 1)
(1, 5)
True
False
True


In [62]:
def permute_demo():
    data0 = torch.randn(4, 5)  # 理论上stride=(5,1)
    data1 = data0.permute(1, 0) #  shape=(5,4) ,理论上stride=(4,1)
#     print(data0.stride())  
#     print(data1.stride())  #(1,5)
    print(data0.is_contiguous())
    print(data1.is_contiguous())  # False

    data2 = data1.contiguous()
    print(data2.is_contiguous())
    
if __name__ == '__main__':
    permute_demo()

True
False
True
